In [9]:
#Sphere Test Function
def sphere(x):#x is a vector  of length n     
  return sum(x**2)

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from ypstruct import structure

In [11]:
# Problem Definition
problem = structure()
problem.costfunc = sphere #definition of costfunction #problem.costfunc(x) = sphere(x)
problem.nvar = 5 # number of variables
problem.varmin = -10 #lower bound of variables  
problem.varmax = 10 #upper bound of variables




In [12]:
# GA Parameters
params = structure()
params.maxit = 100 #maximum number of iterations(generations)
params.npop=20  #population size   (number of individuals)  
params.pc = 1 #crossover percentage (probability of crossover)
params.gamma = 0.1 #extra range factor for crossover    (0.1-0.2)
params.mu = 0.1 #mutation percentage (probability of mutation)  (0.1-0.2)   
params.sigma = 0.1 #mutation step size (gaussian distribution parameter) (0.1-0.2)  


In [13]:
def apply_bound(x,varmin,varmax):#function to apply bounds not to exceed the limits
  x.position = np.maximum(x.position, varmin)
  x.position = np.minimum(x.position, varmax)


In [14]:
def mutate(x, mu, sigma):#function to mutate  (gaussian distribution)
  y = x.deepcopy()
  flag = np.random.rand(*x.position.shape) <= mu
  ind = np.argwhere(flag)
  y.position[ind] += sigma*np.random.randn(*ind.shape)
  return y

In [15]:
def crossover(p1, p2,gamma =0.1) :
#function to perform crossover  (gamma is extra range factor)       
  c1 = p1.deepcopy()
  c2 = p2.deepcopy()
  gamma = 0.1
  alpha = np.random.uniform(-gamma,1+gamma, *c1.position.shape)
  c1.position = alpha*p1.position +(1-alpha)*p2.position
  c2.position = alpha*p2.position +(1-alpha)*p1.position
  return c1, c2

In [16]:
#ga.py

def run(problem, params):#function to run GA algorithm    
  # Problem Information
  costfunc = problem.costfunc
  nvar = problem.nvar
  varmin = problem.varmin
  varmax = problem.varmax
  # Parameters
  maxit = params.maxit
  npop = params.npop
  pc = params.pc
  nc = int(np.round(pc*npop/2)*2)
  gamma = params.gamma
  mu = params.mu
  sigma = params.sigma
  # Empty Individual Template
  empty_individual = structure()
  empty_individual.position = None
  empty_individual.cost = None

  #BestSolution Ever Found
  bestsol = empty_individual.deepcopy()
  bestsol.cost = np.inf


  #Initializiation Population
  pop = empty_individual.repeat(npop)
  for i in range(0, npop):
    pop[i].position = np.random.uniform(varmin,varmax,nvar)
    pop[i].cost = costfunc(pop[i].position)
    if pop[i].cost <bestsol.cost:
      bestsol = pop[i].deepcopy()
  # Best Cost of Iteration
  bestcost= np.empty(maxit)

  # Main Loop
  for it in range(maxit):

    popc = []
    for _ in range(nc//2):

        q = np.random.permutation(npop)
        p1 = pop[q[0]]
        p2 = pop[q[1]]

        # Perform Crossover
        c1, c2 = crossover(p1,p2,gamma)
        # Perform Mutation
        c1 = mutate(c1,mu,sigma)
        c2 = mutate(c2,mu,sigma)

        #Apply Bounds
        apply_bound(c1, varmin, varmax)
        apply_bound(c2, varmin, varmax)

        #Evaluate First Offspring
        c1.cost = costfunc(c1.position)
        if c1.cost<bestsol.cost:
          bestsol = c1.deepcopy()

        #Evaluate Second Offspring
        c2.cost = costfunc(c2.position)
        if c2.cost<bestsol.cost:
          bestsol = c2.deepcopy()

        # Add Offsprings to popc

        popc.append(c1)
        popc.append(c2)

    # Merge, Sort and Select
    pop += popc
    pop = sorted(pop, key= lambda x: x.cost)
    pop = pop[0:npop]

    # Store Best Cost
    bestcost[it] = bestsol.cost

    #Show Iteration Information
    print(f"Iteration {it}:  Best Cost {bestcost[it]}")


  #Output
  out = structure()
  out.pop = pop
  out.bestsol = bestsol
  out.bestcost = bestcost
  return out

In [17]:
#Run GA
out = run(problem,params)



Iteration 0:  Best Cost 16.160768250611977
Iteration 1:  Best Cost 10.90525028085616
Iteration 2:  Best Cost 10.90525028085616
Iteration 3:  Best Cost 3.729062526340078
Iteration 4:  Best Cost 3.729062526340078
Iteration 5:  Best Cost 2.893767187764224
Iteration 6:  Best Cost 1.3108981905994677
Iteration 7:  Best Cost 1.2510602167483378
Iteration 8:  Best Cost 0.6856664844877411
Iteration 9:  Best Cost 0.6856664844877411
Iteration 10:  Best Cost 0.45117275399038465
Iteration 11:  Best Cost 0.44479476023587816
Iteration 12:  Best Cost 0.11972797718838571
Iteration 13:  Best Cost 0.11812240040127894
Iteration 14:  Best Cost 0.09093191252373171
Iteration 15:  Best Cost 0.05600316848168862
Iteration 16:  Best Cost 0.05600316848168862
Iteration 17:  Best Cost 0.03533873395035765
Iteration 18:  Best Cost 0.026812885693316437
Iteration 19:  Best Cost 0.013050206707617152
Iteration 20:  Best Cost 0.01192188507686256
Iteration 21:  Best Cost 0.008388362972486933
Iteration 22:  Best Cost 0.00326

In [18]:
# Result